In [32]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import scipy as sp
import math as mt
from numpy import linalg as LA
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import os,sys
sys.path.append("../Tool/")
import tool as tool

In [113]:
rad2arcsec = 3600*180/np.pi
class Setting():
    def __init__(self,mirror_answer,lens_answer,mirror_data,lens_data):
        self.mirror_answer = mirror_answer
        self.lens_answer   = lens_answer
        self.mirror_data   = mirror_data
        self.lens_data     = lens_data
        self.dist_A_Vertex = 3690.49+19.05 # considering SMR offset
    
    def rotM(self,p):
        # Three dimension rotation (radian)
        px = p[0]
        py = p[1]
        pz = p[2]
        Rx = np.array([[1, 0, 0],
                       [0, np.cos(px), np.sin(px)],
                       [0, -np.sin(px), np.cos(px)]])
        Ry = np.array([[np.cos(py), 0, -np.sin(py)],
                       [0, 1, 0],
                       [np.sin(py), 0, np.cos(py)]])
        Rz = np.array([[np.cos(pz), np.sin(pz), 0],
                       [-np.sin(pz), np.cos(pz), 0],
                       [0, 0, 1]])
        R = Rz.dot(Ry).dot(Rx)
        return R
    
    def calc_R(self,ref_origin=None,ref_move=None):
        """
        calc for Rotation matrix, R.
        This uses mirror answer and data.
        """
        if ref_origin is None and ref_move is None:
            ref_origin = self.mirror_answer
            ref_move   = self.mirror_data
        mean_origin = np.mean(ref_origin,axis=0)
        mean_move   = np.mean(ref_move,axis=0)
        sub_origin  = ref_origin - mean_origin 
        sub_move    = ref_move   - mean_move  
        R, rmsd = align.rotation_matrix(sub_move,sub_origin)
        #print("RMS for calc R:" ,rmsd)
        return R
    def lens_conv(self,data,R):
        """
        Calc new data converted.
        This requires the data we want to convert and the rotation matrix.
        """
        p_origin = self.mirror_answer
        p_move   = self.mirror_data
        #data     = self.lens_data 
        mean_origin = np.mean(p_origin,axis=0)
        mean_move   = np.mean(p_move,axis=0)
        #print("CM-diff:",mean_origin - mean_move)
        sub_data    = data - mean_move
        points = np.dot(R,sub_data.T).T + mean_origin
        return points
    
    def fit_rotz(self,DATA,gamma):
        R = self.calc_R()
        conv_lens_data = self.lens_conv(self.lens_data,R)
        rotz = self.rotM([0,0,gamma])
        lens_points_tmp = np.dot(rotz,DATA.T) # rotated answer
        a = (lens_points_tmp - conv_lens_data.T)
        res = np.abs(a[0]).sum() + np.abs(a[1]).sum() + np.abs(a[2]).sum()
        return res
    def print_result(self,PRINT=True):
        DATA  = self.lens_answer
        param = np.zeros(8) 
        popt,pcov=sp.optimize.curve_fit(self.fit_rotz,DATA,param,p0=[1],method='lm')
        rotz = self.rotM([0,0,popt[0]])
        lens_points = np.dot(rotz,self.lens_answer.T)
        R = self.calc_R()
        conv_lens_data = self.lens_conv(self.lens_data,R)
        diff = np.mean(lens_points-conv_lens_data.T,axis=1)
        if PRINT:
            print("------------------------------------------------------")
            print("LENS_REF_CM ANSWER   :",np.mean(lens_points,axis=1)," [mm]")
            print("LENS_REF_CM DATA     :",np.mean(conv_lens_data.T,axis=1)," [mm]")
            print("LENS_REF CM OFFSET   :",np.mean((lens_points-conv_lens_data.T),axis=1),"   [mm]")
            print("MOVE THE PFC!!       :",np.mean((lens_points-conv_lens_data.T),axis=1),"   [mm]")
            print("MOVE THE PFC!!       :",np.mean((lens_points-conv_lens_data.T),axis=1)/2.5,"   [pitch] per 1-bolt")
            #print("###DIFF STD          :",np.std((lens_points-conv_lens_data.T),axis=1),"   [mm]")
            print("------------------------------------------------------")
        # PFC MC -> vertex vector
        axis      = np.array([0,0,-np.mean(lens_points[2])])
        RotL1     = self.calc_R(conv_lens_data,lens_points.T)
        axis_data = np.dot(RotL1,axis) 
        costheta  = np.dot(axis,axis_data)/np.sqrt(np.dot(axis,axis))/np.sqrt(np.dot(axis_data,axis_data))
        thetax = np.arctan(axis_data[0]/np.abs(axis_data[2]))
        thetay = np.arctan(axis_data[1]/np.abs(axis_data[2]))
        #print(costheta)
        rad_as    = np.arccos(costheta)*rad2arcsec
        if PRINT:
            print("xが正=>mirror中心からM1方向")
            print("yが正=>mirror中心からM7方向")
            print("Axis        : ",axis)
            print("Axis data   : ",axis_data)
            print("tilt angle  : {:.2f}".format(rad_as), "[as]")
            print("tilt angle x: {:.2f}".format(thetax*rad2arcsec), "[as] (Axis from PFC to Mirror)")
            print("tilt angle y: {:.2f}".format(thetay*rad2arcsec), "[as] (Axis from PFC to Mirror)")
            print("ROTATE (%.2f [as] around x-axis) (%.2f [as] around y-axis)"%(-thetay*rad2arcsec,thetax*rad2arcsec))
            radius = 2370
            mm_x = thetax*radius
            mm_y = thetay*radius
            pitch_x = mm_x/1.25/2
            pitch_y = mm_y/1.25/2
            print("ROTATE (%.2f [mm] %.2f [pitch] around x-axis) (%.2f [mm] %.2f [pitch] around y-axis) per 1-bolt"
                  %(-mm_y,-pitch_y,mm_x,pitch_x))
            
            print("------------------------------------------------------")
        return conv_lens_data, lens_points, diff, axis_data    
    def plot_axis(self,save=False):
        conv_lens_data, lens_points, diff, axis_data = self.print_result(False)
        mirror      = go.Scatter3d(x=self.mirror_answer.T[0],y=self.mirror_answer.T[1],z=self.mirror_answer.T[2],name="Mirror_fix_points_ref")
        lens        = go.Scatter3d(x=lens_points.T[0],y=lens_points.T[1],z=lens_points.T[2],name="Lens_fix_points_ref")
        lens_data   = go.Scatter3d(x=conv_lens_data.T[0],y=conv_lens_data.T[1],z=conv_lens_data.T[2],name="Lens_fix_points_data")
        axis        = go.Scatter3d(x=[0,0],y=[0,0],z=[0,np.mean(lens_points[2])],name="Optical_axis")
        tmpx0 = -diff[0]
        tmpx1 = -diff[0]+axis_data[0]
        tmpy0 = -diff[1]
        tmpy1 = -diff[1]+axis_data[1]
        tmpz0 = -diff[2]+np.mean(lens_points[2])
        tmpz1 = -diff[2]+axis_data[2]+np.mean(lens_points[2])
        axis_2 = go.Scatter3d(x=[tmpx0,tmpx1],y=[tmpy0,tmpy1],z=[tmpz0,tmpz1],name="Lens_axis_guess")
        vertex = go.Scatter3d(x=[0],y=[0],z=[0],name="vertex")
        data0 = [axis,axis_2]
        #data0 = [lens,lens_data]
        layout = go.Layout(
            width=1200*0.6,
            height=1000*0.6,
            scene = dict(
                #xaxis = dict(range = [-30,+30]),
                #yaxis = dict(range = [-30,+30]),
                #zaxis = dict(range = [-500,+3500])
            )   
            ,showlegend = True
        )
        fig = go.Figure(data=data0,layout=layout)
        fig.update_layout(scene_aspectmode='cube')
        if save is True:
            fig.write_html("./figure_axis.html")
        fig.show()
    def plot_result(self,save=False):
        conv_lens_data, lens_points, diff, axis_data = self.print_result(False)
        mirror      = go.Scatter3d(x=self.mirror_answer.T[0],y=self.mirror_answer.T[1],z=self.mirror_answer.T[2],name="Mirror_fix_answer")
        lens        = go.Scatter3d(x=lens_points[0],y=lens_points[1],z=lens_points[2],name="Lens_fix_answer")
        lens_data   = go.Scatter3d(x=conv_lens_data.T[0],y=conv_lens_data.T[1],z=conv_lens_data.T[2],name="Lens_fix_data")
        axis        = go.Scatter3d(x=[0,0],y=[0,0],z=[0,np.mean(lens_points[2])],name="Mirror_axis")
        
        tmpx0 = -diff[0]
        tmpx1 = -diff[0]+axis_data[0]
        tmpy0 = -diff[1]
        tmpy1 = -diff[1]+axis_data[1]
        tmpz0 = -diff[2]+np.mean(lens_points[2])
        tmpz1 = -diff[2]+axis_data[2]+np.mean(lens_points[2])
        axis_2 = go.Scatter3d(x=[tmpx1,tmpx0],y=[tmpy1,tmpy0],z=[tmpz1,tmpz0],name="Lens_axis_guess")
        vertex = go.Scatter3d(x=[0],y=[0],z=[0],name="vertex")
        data0 = [axis,axis_2,mirror,lens,lens_data]
        layout = go.Layout(
            width=1200*0.6,
            height=1000*0.6,
            scene = dict(
                #xaxis = dict(range = [-30,+30]),
                #yaxis = dict(range = [-30,+30]),
                #zaxis = dict(range = [-500,+3500])
            )   
            ,showlegend = True
        )
        fig = go.Figure(data=data0,layout=layout)
        fig.update_layout(scene_aspectmode='cube')
        if save is True:
            fig.write_html("./figure_result.html")
        fig.show()

In [114]:
np.set_printoptions(formatter={'float': '{:.4f}'.format})
mirror_answer = np.load("../notebooks/mirror_answer1.npy")
lens_answer = np.load("../notebooks/lens_answer1.npy")

In [115]:
def data_reduction(data):
    a = np.array(data)
    b = a.reshape(8,5,3)
    c = np.mean(b,axis=1)
    return c

mirror_data = pd.read_csv("./data/polyline2-3.txt",names=["x","y","z"],usecols=[0,1,2],comment="#")
lens_data   = pd.read_csv("./data/polyline1.txt",names=["x","y","z"],usecols=[0,1,2],comment="#")

m1 = data_reduction(mirror_data)
l1 = data_reduction(lens_data)

In [116]:
Set1 = Setting(mirror_answer,lens_answer,m1,l1)
Set1.print_result();
Set1.plot_result(save=True)
Set1.plot_axis(save=True)

------------------------------------------------------
LENS_REF_CM ANSWER   : [1.4576 2.5833 3043.2129]  [mm]
LENS_REF_CM DATA     : [4.1160 -0.3010 3028.7691]  [mm]
LENS_REF CM OFFSET   : [-2.6584 2.8844 14.4438]    [mm]
MOVE THE PFC!!       : [-2.6584 2.8844 14.4438]    [mm]
MOVE THE PFC!!       : [-1.0634 1.1538 5.7775]    [pitch] per 1-bolt
------------------------------------------------------
xが正=>mirror中心からM1方向
yが正=>mirror中心からM7方向
Axis        :  [0.0000 0.0000 -3043.2129]
Axis data   :  [-9.7028 3.2083 -3043.1958]
tilt angle  : 692.66 [as]
tilt angle x: -657.64 [as] (Axis from PFC to Mirror)
tilt angle y: 217.46 [as] (Axis from PFC to Mirror)
ROTATE (-217.46 [as] around x-axis) (-657.64 [as] around y-axis)
ROTATE (-2.50 [mm] -1.00 [pitch] around x-axis) (-7.56 [mm] -3.02 [pitch] around y-axis) per 1-bolt
------------------------------------------------------
